# Data Munging

In [22]:
#Dependencies
import pandas as pd
import numpy as py
import matplotlib.pyplot as plt
from pprint import pprint
import json

#Importing NLP model
from score import score

In [28]:
#Open and read movie data and reviews data
with open("./movie_data.json", "r") as io:
    movie_data = json.loads(io.read())
    io.close()

with open("./reviews.json", "r") as io:
    reviews = json.loads(io.read())
    io.close()

In [52]:
#Loop through key,value pairs
movie_ids = [str(x) for x in movie_data["movie_ids"]]

m_reviews = []

for movie_id in movie_ids:
    review_list = reviews[movie_id]
    for review in review_list:
        row = {}
        row["movie_id"] = movie_id
        row["review"] = review
        index = movie_data["movie_ids"].index(int(movie_id))
        row["title"] = movie_data["titles"][index]
        row["date"] = movie_data["dates"][index]
        row["average_score"] = movie_data["average_score"][index]
        row["vote_count"] = movie_data["vote_count"][index]
        row["popularity"] = movie_data["popularity"][index]
        m_reviews.append(row)

In [58]:
#Convert to dataframe
reviews_df = pd.DataFrame(m_reviews)

In [66]:
#Run NLP on each review
review_series = reviews_df.review
scores = review_series.map(lambda r : score(r))

In [67]:
#Add scores to dataframe
reviews_df["scores"] = scores

In [68]:
reviews_df.head()

,movie_id,review,title,date,average_score,vote_count,popularity,scores
0,24428,With a movie like this you wonder how all of t...,The Avengers,2012-04-25,7.7,21240,27.884,0.176025
1,24428,**A long format review from 2012**\r\n\r\nMy n...,The Avengers,2012-04-25,7.7,21240,27.884,12.144836
2,24428,"After the recent disappointments from Marvel, ...",The Avengers,2012-04-25,7.7,21240,27.884,-0.390126
3,24428,One of the few movies to actually make me feel...,The Avengers,2012-04-25,7.7,21240,27.884,1.961338
4,24428,I give this 10 Stars based on it being not onl...,The Avengers,2012-04-25,7.7,21240,27.884,0.701263
